In [140]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [141]:
out='../output/'
inp='../input/monthly/'
pref='fpm'

In [142]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
744,(2021/05/31) 23-24,39.95,20
745,Min,39.95,0
746,Max,100,20
747,Average,69.329301075268817204301075269,10.215053763440860215053763441


In [143]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/04/01) 04-05,39.06,9.4
6,(2017/04/01) 05-06,39.06,9.4
7,(2017/04/01) 06-07,39.06,9.4
8,(2017/04/01) 07-08,67.32,0.0
9,(2017/04/01) 08-09,67.32,0.0
...,...,...,...
740,(2021/05/31) 19-20,100.00,0.0
741,(2021/05/31) 20-21,100.00,0.0
742,(2021/05/31) 21-22,100.00,0.0
743,(2021/05/31) 22-23,39.95,20.0


In [144]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/04/01) 04-05,39.06,9.4,2017-04-01,4
6,(2017/04/01) 05-06,39.06,9.4,2017-04-01,5
7,(2017/04/01) 06-07,39.06,9.4,2017-04-01,6
8,(2017/04/01) 07-08,67.32,0.0,2017-04-01,7
9,(2017/04/01) 08-09,67.32,0.0,2017-04-01,8
...,...,...,...,...,...
740,(2021/05/31) 19-20,100.00,0.0,2021-05-31,19
741,(2021/05/31) 20-21,100.00,0.0,2021-05-31,20
742,(2021/05/31) 21-22,100.00,0.0,2021-05-31,21
743,(2021/05/31) 22-23,39.95,20.0,2021-05-31,22


In [145]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,39.06,9.4,2017-04-01,4
6,39.06,9.4,2017-04-01,5
7,39.06,9.4,2017-04-01,6
8,67.32,0.0,2017-04-01,7
9,67.32,0.0,2017-04-01,8
...,...,...,...,...
740,100.00,0.0,2021-05-31,19
741,100.00,0.0,2021-05-31,20
742,100.00,0.0,2021-05-31,21
743,39.95,20.0,2021-05-31,22


In [146]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-04-01,39.06,9.4,4,2017,4,1,5
2017-04-01,39.06,9.4,5,2017,4,1,5
2017-04-01,39.06,9.4,6,2017,4,1,5
2017-04-01,67.32,0.0,7,2017,4,1,5
2017-04-01,67.32,0.0,8,2017,4,1,5
...,...,...,...,...,...,...,...
2021-05-31,100.00,0.0,19,2021,5,31,0
2021-05-31,100.00,0.0,20,2021,5,31,0
2021-05-31,100.00,0.0,21,2021,5,31,0


In [147]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.groupby(['year','month','day']).sum().drop(['hour'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume  day_of_week      value  price_weight_av  price_max  \
year month day                                                               
2017 4     1     122.2          100   4773.132            39.06      67.32   
           2     225.6          144   8811.936            39.06      39.06   
           3      37.6            0   1468.656            39.06      67.32   
           4      37.6           20   1468.656            39.06      67.32   
           5      75.2           48   2937.312            39.06      67.32   
...                ...          ...        ...              ...        ...   
2021 5     27    160.0           72   6392.000            39.95     100.00   
           28    160.0           96   6392.000            39.95     100.00   
           29    260.0          100  10387.000            39.95     100.00   
           30    400.0          120  15980.000            39.95      39.95   
           31     80.0            0   3196.000            39.95     100.00   

                price_min  price_mean  power_max  power_min  power_mean  \
year month day                                                            
2017 4     1        39.06   48.951000        9.4        0.0    6.110000   
           2        39.06   39.060000        9.4        9.4    9.400000   
           3        39.06   61.668000        9.4        0.0    1.880000   
           4        39.06   61.668000        9.4        0.0    1.880000   
           5        39.06   57.900000        9.4        0.0    3.133333   
...                   ...         ...        ...        ...         ...   
2021 5     27       39.95   79.983333       20.0        0.0    6.666667   
           28       39.95   79.983333       20.0        0.0    6.666667   
           29       39.95   60.967500       20.0        0.0   13.000000   
           30       39.95   39.950000       20.0       20.0   20.000000   
           31       39.95   87.990000       20.0        0.0    4.000000   

                price_morning_max  price_evening_max  price_standard  
year month day                                                        
2017 4     1                67.32              67.32           39.06  
           2                39.06              39.06           39.06  
           3                67.32              67.32           67.32  
           4                67.32              67.32           67.32  
           5                67.32              67.32           67.32  
...                           ...                ...             ...  
2021 5     27              100.00             100.00          100.00  
           28              100.00             100.00          100.00  
           29              100.00             100.00           39.95  
           30               39.95              39.95           39.95  
           31              100.00             100.00          100.00  

[1388 rows x 13 columns]

In [148]:
hourly['cat']='off'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-04-01,39.06,9.4,4,2017,4,1,5,367.164,off
2017-04-01,39.06,9.4,5,2017,4,1,5,367.164,off
2017-04-01,39.06,9.4,6,2017,4,1,5,367.164,off
2017-04-01,67.32,0.0,7,2017,4,1,5,0.000,standard
2017-04-01,67.32,0.0,8,2017,4,1,5,0.000,standard
...,...,...,...,...,...,...,...,...,...
2021-05-31,100.00,0.0,19,2021,5,31,0,0.000,evening
2021-05-31,100.00,0.0,20,2021,5,31,0,0.000,standard
2021-05-31,100.00,0.0,21,2021,5,31,0,0.000,standard


In [149]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume  day_of_week      value  price_weight_av  price_max  \
year month day                                                               
2017 4     1     122.2          100   4773.132            39.06      67.32   
           2     225.6          144   8811.936            39.06      39.06   
           3      37.6            0   1468.656            39.06      67.32   
           4      37.6           20   1468.656            39.06      67.32   
           5      75.2           48   2937.312            39.06      67.32   
...                ...          ...        ...              ...        ...   
2021 5     27    160.0           72   6392.000            39.95     100.00   
           28    160.0           96   6392.000            39.95     100.00   
           29    260.0          100  10387.000            39.95     100.00   
           30    400.0          120  15980.000            39.95      39.95   
           31     80.0            0   3196.000            39.95     100.00   

                price_min  price_mean  power_max  power_min  power_mean  ...  \
year month day                                                           ...   
2017 4     1        39.06   48.951000        9.4        0.0    6.110000  ...   
           2        39.06   39.060000        9.4        9.4    9.400000  ...   
           3        39.06   61.668000        9.4        0.0    1.880000  ...   
           4        39.06   61.668000        9.4        0.0    1.880000  ...   
           5        39.06   57.900000        9.4        0.0    3.133333  ...   
...                   ...         ...        ...        ...         ...  ...   
2021 5     27       39.95   79.983333       20.0        0.0    6.666667  ...   
           28       39.95   79.983333       20.0        0.0    6.666667  ...   
           29       39.95   60.967500       20.0        0.0   13.000000  ...   
           30       39.95   39.950000       20.0       20.0   20.000000  ...   
           31       39.95   87.990000       20.0        0.0    4.000000  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2017 4     1                   NaN                 NaN       122.2   
           2                   NaN                 NaN       225.6   
           3                   3.0                 2.0        37.6   
           4                   3.0                 2.0        37.6   
           5                   3.0                 2.0        75.2   
...                            ...                 ...         ...   
2021 5     27                  3.0                 2.0       160.0   
           28                  3.0                 2.0       160.0   
           29                  NaN                 NaN       260.0   
           30                  NaN                 NaN       400.0   
           31                  3.0                 2.0        80.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2017 4     1                0.0             NaN             NaN   4773.132   
           2                NaN             NaN             NaN   8811.936   
           3                0.0             0.0             0.0   1468.656   
           4                0.0             0.0             0.0   1468.656   
           5                0.0             0.0             0.0   2937.312   
...                         ...             ...             ...        ...   
2021 5     27               0.0             0.0             0.0   6392.000   
           28               0.0             0.0             0.0   6392.000   
           29               0.0             NaN             NaN  10387.000   
           30               NaN             NaN             NaN  15980.000   
           31               0.0             0.0             0.0   3196.000   

                standard_value  mo

In [150]:
daily.to_csv(out + pref + '_daily.csv')

In [151]:
hourly.to_csv(out + pref + '_hourly.csv')

In [152]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,4,105.593333,73.200000,4124.475600,61.668,39.06,53.606050,9.400000,1.566667,...,40.0,39.060000,39.06,NaN,NaN,NaN,9.400000,0.000000,0.000000,0.000000
1,2017,5,107.096774,65.290323,3970.077419,57.678,37.07,50.891763,10.000000,1.612903,...,46.0,37.070000,37.07,37.070000,37.07,37.07,10.000000,0.434783,0.391459,0.434783
2,2017,6,26.400000,78.000000,1176.384000,68.368,64.40,66.384000,4.400000,0.000000,...,6.0,44.560000,44.56,NaN,NaN,NaN,4.400000,0.000000,0.000000,0.000000
3,2017,7,273.333333,81.333333,11540.133333,68.404,42.22,54.766500,20.000000,6.666667,...,3.0,42.220000,42.22,42.220000,NaN,NaN,20.000000,0.000000,1.111111,0.000000
4,2017,8,380.833333,78.666667,22038.583333,75.400,33.40,62.785417,20.000000,15.833333,...,12.0,57.869365,33.40,84.451724,85.90,85.90,20.000000,15.000000,15.104167,15.000000
5,2017,9,214.000000,70.666667,6623.300000,56.446,30.95,48.000450,20.000000,2.666667,...,42.0,30.950000,30.95,NaN,NaN,NaN,20.000000,0.000000,0.000000,0.000000
6,2017,10,214.193548,67.612903,6237.316129,54.248,29.12,45.897331,20.000000,3.225806,...,44.0,29.120000,29.12,NaN,NaN,NaN,20.000000,0.000000,0.000000,0.000000
7,2017,11,334.666667,68.400000,12981.280000,51.628,27.26,44.207606,20.000000,11.333333,...,44.0,38.788685,27.26,57.720000,57.72,57.72,20.000000,10.000000,10.000000,10.000000
8,2017,12,322.922581,72.258065,11881.128000,51.972,26.74,43.069581,20.000000,10.438710,...,42.0,36.792497,26.74,58.280000,58.28,58.28,20.000000,8.600000,8.600000,8.600000
9,2018,1,214.193548,65.290323,6331.561290,55.256,29.56,46.794280,20.000000,3.225806,...,46.0,29.560000,29.56,29.560000,29.56,29.56,19.746835,0.869565,1.067616,0.869565


In [153]:
monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power,evening_days,morning_days,days
0,2017,4,105.593333,73.200000,4124.475600,61.668,39.06,53.606050,9.400000,1.566667,...,NaN,NaN,NaN,9.400000,0.000000,0.000000,0.000000,20.0,20.0,30
1,2017,5,107.096774,65.290323,3970.077419,57.678,37.07,50.891763,10.000000,1.612903,...,37.070000,37.07,37.07,10.000000,0.434783,0.391459,0.434783,23.0,23.0,31
2,2017,6,26.400000,78.000000,1176.384000,68.368,64.40,66.384000,4.400000,0.000000,...,NaN,NaN,NaN,4.400000,0.000000,0.000000,0.000000,2.0,2.0,2
3,2017,7,273.333333,81.333333,11540.133333,68.404,42.22,54.766500,20.000000,6.666667,...,42.220000,NaN,NaN,20.000000,0.000000,1.111111,0.000000,1.0,1.0,3
4,2017,8,380.833333,78.666667,22038.583333,75.400,33.40,62.785417,20.000000,15.833333,...,84.451724,85.90,85.90,20.000000,15.000000,15.104167,15.000000,4.0,4.0,6
5,2017,9,214.000000,70.666667,6623.300000,56.446,30.95,48.000450,20.000000,2.666667,...,NaN,NaN,NaN,20.000000,0.000000,0.000000,0.000000,21.0,21.0,30
6,2017,10,214.193548,67.612903,6237.316129,54.248,29.12,45.897331,20.000000,3.225806,...,NaN,NaN,NaN,20.000000,0.000000,0.000000,0.000000,22.0,22.0,31
7,2017,11,334.666667,68.400000,12981.280000,51.628,27.26,44.207606,20.000000,11.333333,...,57.720000,57.72,57.72,20.000000,10.000000,10.000000,10.000000,22.0,22.0,30
8,2017,12,322.922581,72.258065,11881.128000,51.972,26.74,43.069581,20.000000,10.438710,...,58.280000,58.28,58.28,20.000000,8.600000,8.600000,8.600000,21.0,21.0,31
9,2018,1,214.193548,65.290323,6331.561290,55.256,29.56,46.794280,20.000000,3.225806,...,29.560000,29.56,29.56,19.746835,0.869565,1.067616,0.869565,23.0,23.0,31


In [154]:
monthly.to_csv(out + pref + '_monthly.csv')

In [155]:
monthly_filt=monthly.loc[monthly['year']<2020]
daily_filt=daily.reset_index().loc[daily.reset_index()['year']<2020]

In [156]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]

calmonthly

,month,day_of_week,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,off_value,standard_value,morning_value,evening_value,volume,off_volume,standard_volume,morning_volume,evening_volume,value
0,1,66.387097,3165.780645,29.560000,77.092742,26.755000,58.744672,10.000000,1.612903,4.634409,...,92227.200000,4434.000000,886.800,591.200000,3320.000000,3120.000000,150.000000,30.0,20.000000,98139.200000
1,2,69.142857,3390.685714,31.230000,78.667143,27.950000,59.427378,10.000000,1.428571,4.684524,...,93690.000000,1249.200000,0.000,0.000000,3040.000000,3000.000000,40.000000,0.0,0.000000,94939.200000
2,3,71.161290,3580.500000,32.550000,59.247581,27.775000,47.194651,10.000000,1.612903,4.827957,...,104160.000000,5208.000000,976.500,651.000000,3410.000000,3200.000000,160.000000,30.0,20.000000,110995.500000
3,4,69.511111,3799.642978,35.390000,48.276667,31.893333,42.107943,9.800000,1.855556,4.833500,...,110605.822667,2326.133333,634.400,422.933333,3349.266667,3242.600000,73.333333,20.0,13.333333,113989.289333
4,5,67.182796,2464.296774,34.515000,59.628817,34.160000,50.517714,6.666667,1.075269,3.075269,...,72711.600000,2531.100000,690.300,460.200000,2213.333333,2106.666667,73.333333,20.0,13.333333,76393.200000
5,6,72.129032,37.947871,44.560000,82.694032,42.126129,67.289200,0.141935,0.000000,0.037849,...,784.256000,0.000000,0.000,0.000000,17.600000,17.600000,0.000000,0.0,0.000000,784.256000
6,7,67.138462,29615.474462,69.988000,132.806923,42.272462,100.441146,29.806154,1.846154,16.588308,...,156858.666667,317148.474667,83830.736,83830.736000,8277.333333,2866.666667,3545.066667,932.8,932.800000,641668.613333
7,8,70.235294,1944.580882,60.402107,101.647206,42.722794,80.283048,1.764706,1.397059,1.517770,...,13360.000000,20409.166667,5154.000,5154.000000,761.666667,400.000000,241.666667,60.0,60.000000,44077.166667
8,9,71.022222,4082.984444,27.995000,64.234111,33.980000,52.801049,13.333333,2.000000,6.335185,...,122489.533333,0.000000,0.000,0.000000,4386.666667,4386.666667,0.000000,0.0,0.000000,122489.533333
9,10,66.795699,4455.561290,32.045000,70.931828,30.676667,56.684361,13.333333,1.935484,5.994624,...,138122.400000,0.000000,0.000,0.000000,4320.000000,4320.000000,0.000000,0.0,0.000000,138122.400000


In [157]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [158]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                   
1     0     36.237500  15.000000   605.000000
      1     36.237500  15.000000   605.000000
      2     36.237500  15.000000   605.000000
      3     36.237500  15.000000   605.000000
      4     36.237500  15.000000   605.000000
...               ...        ...          ...
12    19    77.840645   7.027957   362.526968
      20    72.651828   9.576344   470.849118
      21    72.651828   9.576344   470.849118
      22    43.920000  23.333333  1078.666667
      23    43.920000  23.333333  1078.666667

[288 rows x 3 columns]

In [159]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')